In [275]:
from tqdm import tqdm
from bitstring import BitArray
import numpy as np
from collections import *

In [209]:
def fibonacci_hash_float(value:float):
    
    value = BitArray(float=value, length=64)
    phi = (1 + 5 ** 0.5) / 2
    g = int(2 ** 64 /phi)
    

    value ^= value >> 61
    value = int(g * value.float * np.random.random_sample())
 
    return value

In [116]:
def minhash(freqs, times, threshold = None):
    old_threshold = threshold
    picks = []
    out = []
    f = 0
    
    for i in range(0,threshold):
        picks = []
        for fr, tm in zip(freqs,times):
            picks.append(fibonacci_hash_float(fr) ^ fibonacci_hash_float(tm))
        
        out.append(np.min(picks))
        


    
    return out    

In [119]:
b = 30
a = 0
es = (b - a) * np.random.random_sample(50) + a

In [118]:
minhash(np.random.rand(50), es, 5)

[703138831747335424,
 240150506633018624,
 1852260760376621312,
 635893337723965952,
 68715354666240000]

In [124]:
tes = []

tes.append(minhash(np.random.rand(50), es, 6))
tes.append(minhash(np.random.rand(50), es, 6))

In [151]:
m = np.array(tes).T
m

array([[ 839062061416448896,  115851297132361216],
       [1370521585378371488, 1063048003586958816],
       [ 282878549659531264,  242231118334144928],
       [  38858651052930432,  323474749746183936],
       [ 445912721613501696,  910479426537176064],
       [ 557524212649909248, 1881576893128818944]])

In [ ]:
def make_signatureMatrix(songs):
    sig_matrix = []
    for freq, tm in song:
        sig_matrix = minhash(freq,tm, 5)
        
    return sig_matrix.T

In [229]:
m

array([[ 839062061416448896,  115851297132361216],
       [1370521585378371488, 1063048003586958816],
       [ 282878549659531264,  242231118334144928],
       [  38858651052930432,  323474749746183936],
       [ 445912721613501696,  910479426537176064],
       [ 557524212649909248, 1881576893128818944]])

In [293]:
def lsh(sig_matrix, b=2):

    band_numb = sig_matrix.shape[0]//b
    bucket_matrix = []

    for i in range(0,band_numb):

        band = m[b*i:b*(i+1)]
        
        bucket_row = defaultdict(lambda: 0)
        
        for j in range(band.shape[1]):
            
            bucket_row[hash_2(band[:,j])] += 1

        bucket_matrix.append(bucket_row.values())


    return np.array(bucket_matrix)


In [294]:
lsh(m)

array([dict_values([1, 1]), dict_values([1, 1]), dict_values([1, 1])],
      dtype=object)

In [284]:
di = defaultdict(lambda: 0)

di["ciao"]

0

In [267]:
def hash_2(vet):
    out = 0
    for el in vet:
        out ^= el
        
    out = fibonacci_hash_float(out)
        
    return int(str(out)[:15])

In [268]:
hash_2(m[0:b*2,][:,0])

183673185045734